If all samples were actually equally likely, then the sum of $1$s drawn from the population is distributed as hypergeometric with $p = n\times \text{popMean}$ "good" items, $n-p = n*(1-\text{popMean})$ "bad" items, and $k$ draws. This random variable $X$ is distributed with
$$E(X) = k\text{popMean} = \frac{kp}{n}$$
and $$var(X) = \frac{k\text{popMean}(1-\text{popMean})(n-k)}{n-1} = \frac{kp(1-p)(n-k)}{n^2(n-1)}.$$
Instead of the sum, we look at the mean of the $k$ draws, $\bar{X}$. It has a scaled hypergeometric distribution, with
$$E(\bar{X}) = \text{popMean} = \frac{p}{n}$$
and $$var(\bar{X}) = \frac{\text{popMean}(1-\text{popMean})(n-k)}{k(n-1)} = \frac{p(1-p)(n-k)}{kn^2(n-1)}.$$
Finally, we sample from this distribution $B$ times and take the sample average -- this is the column Sample Mean in the table. This is an average of IID random variables, so it has mean $\text{popMean} = \frac{p}{n}$ and variance
$$\frac{\text{popMean}(1-\text{popMean})(n-k)}{B^2k(n-1)} = \frac{p(1-p)(n-k)}{B^2kn^2(n-1)}.$$
For large $B$, this is miniscule. If $B = 10^4$, the variance should be on the order of $10^{-8}$. Yet, we see bias on the order of $10^{-3}$.

In [1]:
getEmpiricalDistr <- function(n, k, reps=1e7){
  samReplicates <- t(replicate(reps, sort(sample(n, k))))
  uniqueSampleVec <- unique(samReplicates)
  uniqueSamples <- vector("list", nrow(uniqueSampleVec))
  for(i in seq_along(uniqueSamples)){
    sam <- uniqueSampleVec[i, ]
    uniqueSamples[[i]]$sample <- sam
    uniqueSamples[[i]]$freq <- sum(apply(samReplicates, 1, function(row) all(row==sam)))
  }
  return(uniqueSamples)
}

                                         
getEmpiricalDistrRaw <- function(n, k, reps=1e7){
    # Just generates the samples and leaves them in matrix form,
    # one per row, unsorted.
    samReplicates <- t(replicate(reps, sort(sample(n, k))))
    return(samReplicates)
}
    

makePopulation <- function(n, p){
    # Create a population of 0s and 1s
    # n = pop size
    # p = number of 1s in the population

    x <- rep(0, n)
    x[1:p] <- 1
    return(x)
}
                      

getPopMean <- function(x){
    return(mean(x))
}


getSampleMean <- function(x, samReplicates){
    # Takes input:
    # x = population
    # samReplicates = samples. output from getEmpiricalDistrRaw, not getEmpiricalDistr
    sampMeans <- apply(samReplicates, 1, function(sam) mean(x[sam]))
    return(mean(sampMeans))
}

# Super-Duper

In [2]:
# Boilerplate stuff

reps <- 1e4
n <- c(13, 30, 90)
k <- c(4, 10, 20)
p <- c(5, 10, 20)

popMean <- c()
sampleMean <- c()
nvalues <- c()
kvalues <- c()
prng <- c()
seed <- c()
bias <- c()
relBias <- c()
theoreticalSE <- c()

In [3]:
seedvalues = c(100, 233424280)

for(nn in n){
  for(kk in k){
    for(pp in p){
      if(pp >= nn | kk >= nn){
        next
      }
      for(ss in seedvalues){
        set.seed(ss, kind = "Super-Duper")
        
        itemCounts <- getEmpiricalDistrRaw(n=nn, k=kk, reps=reps)
        
        x <- makePopulation(nn, pp)
        truePopMean <- getPopMean(x)
        popMean <- c(popMean, truePopMean)
        obsSampMean <- getSampleMean(x, itemCounts)
        sampleMean <- c(sampleMean, obsSampMean)
        nvalues <- c(nvalues, nn)
        kvalues <- c(kvalues, kk)
        prng <- c(prng, "Super-Duper")
        seed <- c(seed, ss)
        
        estimBias <- obsSampMean - truePopMean
        bias <- c(bias, estimBias)
        relBias <- c(relBias, estimBias/truePopMean)
        
        theoreticalSE <- c(theoreticalSE,
                           sqrt(truePopMean*(1-truePopMean)*(nn-kk)/(reps^2 * kk * nn^2 * (nn-1))))
      }
    }
  }
}

In [4]:
cbind("Pop size" = nvalues, 
      "Sample size" = kvalues, 
      "seed" = seedvalues, 
      "Pop Mean" = popMean, 
      "Sample Mean" = sampleMean, 
      "Bias" = bias, 
      "Relative bias" = relBias, 
      "Theoretical SE" = theoreticalSE
)

Pop size,Sample size,seed,Pop Mean,Sample Mean,Bias,Relative bias,Theoretical SE
13,4,100,0.38461538,0.385900,1.284615e-03,0.0033400,1.620481e-06
13,4,233424280,0.38461538,0.382600,-2.015385e-03,-0.0052400,1.620481e-06
13,4,100,0.76923077,0.771150,1.919231e-03,0.0024950,1.403378e-06
13,4,233424280,0.76923077,0.767300,-1.930769e-03,-0.0025100,1.403378e-06
13,10,100,0.38461538,0.384840,2.246154e-04,0.0005840,5.917160e-07
13,10,233424280,0.38461538,0.386080,1.464615e-03,0.0038080,5.917160e-07
13,10,100,0.76923077,0.770230,9.992308e-04,0.0012990,5.124411e-07
13,10,233424280,0.76923077,0.769430,1.992308e-04,0.0002590,5.124411e-07
30,4,100,0.16666667,0.168150,1.483333e-03,0.0089000,5.881257e-07
30,4,233424280,0.16666667,0.165200,-1.466667e-03,-0.0088000,5.881257e-07


# Mersenne Twister

In [5]:
# Boilerplate stuff

reps <- 1e4
n <- c(13, 30, 90)
k <- c(4, 10, 20)
p <- c(5, 10, 20)

popMean <- c()
sampleMean <- c()
nvalues <- c()
kvalues <- c()
prng <- c()
seed <- c()
bias <- c()
relBias <- c()
theoreticalSE <- c()

In [6]:
seedvalues = c(100, 233424280, 429496729)

for(nn in n){
  for(kk in k){
    for(pp in p){
      if(pp >= nn | kk >= nn){
        next
      }
      for(ss in seedvalues){
        set.seed(ss, kind = "Mersenne-Twister")
        
        itemCounts <- getEmpiricalDistrRaw(n=nn, k=kk, reps=reps)
        
        x <- makePopulation(nn, pp)
        truePopMean <- getPopMean(x)
        popMean <- c(popMean, truePopMean)
        obsSampMean <- getSampleMean(x, itemCounts)
        sampleMean <- c(sampleMean, obsSampMean)
        nvalues <- c(nvalues, nn)
        kvalues <- c(kvalues, kk)
        prng <- c(prng, "Mersenne Twister")
        seed <- c(seed, ss)
        
        estimBias <- obsSampMean - truePopMean
        bias <- c(bias, estimBias)
        relBias <- c(relBias, estimBias/truePopMean)
        
        theoreticalSE <- c(theoreticalSE,
                           sqrt(truePopMean*(1-truePopMean)*(nn-kk)/(reps^2 * kk * nn^2 * (nn-1))))
      }
    }
  }
}

In [7]:
cbind("Pop size" = nvalues, 
      "Sample size" = kvalues, 
      "seed" = seedvalues, 
      "Pop Mean" = popMean, 
      "Sample Mean" = sampleMean, 
      "Bias" = bias, 
      "Relative bias" = relBias, 
      "Theoretical SE" = theoreticalSE
)

Pop size,Sample size,seed,Pop Mean,Sample Mean,Bias,Relative bias,Theoretical SE
13,4,100,0.3846154,0.388100,3.484615e-03,0.0090600,1.620481e-06
13,4,233424280,0.3846154,0.384525,-9.038462e-05,-0.0002350,1.620481e-06
13,4,429496729,0.3846154,0.383875,-7.403846e-04,-0.0019250,1.620481e-06
13,4,100,0.7692308,0.768075,-1.155769e-03,-0.0015025,1.403378e-06
13,4,233424280,0.7692308,0.770775,1.544231e-03,0.0020075,1.403378e-06
13,4,429496729,0.7692308,0.768800,-4.307692e-04,-0.0005600,1.403378e-06
13,10,100,0.3846154,0.384560,-5.538462e-05,-0.0001440,5.917160e-07
13,10,233424280,0.3846154,0.384430,-1.853846e-04,-0.0004820,5.917160e-07
13,10,429496729,0.3846154,0.384030,-5.853846e-04,-0.0015220,5.917160e-07
13,10,100,0.7692308,0.768680,-5.507692e-04,-0.0007160,5.124411e-07
